In [48]:
import pandas
import numpy as np
import bar_chart_race as bcr

from math import factorial
from collections import defaultdict
from warnings import filterwarnings

filterwarnings("ignore")

In [49]:
N = 5
M = 12
LMD = 10.5
MU = 0.3
P = 0.7

TICK_TIME = 0.01

In [50]:
class Request:
    total_requests = 0

    def __init__(self, *, time):
        Request.total_requests += 1
        self.start_queue_time = time
        self.in_queue_time = 0
        self.working_time = 0

    def tick_in_queue(self):
        self.in_queue_time += TICK_TIME

    def tick_working(self):
        self.working_time += TICK_TIME

In [51]:
class Channel:
    request = None

    def _get_executing_time(self):
        return np.random.exponential(1 / self.mu)

    def __init__(self, *, mu, callback):
        self.mu = mu
        self.callback = callback
        self.executing_time = 0

    def push(self, request):
        self.request = request
        self.executing_time = self._get_executing_time()

    def try_finish(self):
        if self.executing_time <= 0:
            self.callback(self)

    def tick(self):
        self.executing_time -= TICK_TIME
        self.request.tick_working()
        self.try_finish()

In [52]:
class SMO:
    def _should_accept(self):
        return np.random.uniform() <= self.p

    def _get_requests_amount(self):
        return round(np.random.exponential(self.lmd))

    def __init__(self, *, n, m, lmd, mu, p):
        self.n = n
        self.m = m
        self.lmd = lmd
        self.mu = mu
        self.p = p
        self.q = 1 - p
        
        # Stats ##################
        self.accepted_requests = 0
        self.rejected_requests = 0
        self.cancelled_requests = 0
        self.request_work_time = []
        self.request_queue_time = []
        self.avg_requests = []
        self.busy_channels = []
        self.queueing_requests = []
        ##########################

        self.free_channels = []
        self.working_channels = []
        self.requests_queue = []

        self.current_time = 0

        for _ in range(n):
            self.free_channels.append(Channel(mu=self.mu, callback=self._channel_finishing_callback))

    def simulate(self):
        if abs(self.current_time - int(self.current_time)) < TICK_TIME:
            cnt = self._get_requests_amount()
            req_cnt = min(cnt, self.m - len(self.requests_queue))
            self.cancelled_requests += cnt - req_cnt
            for _ in range(req_cnt):
                self.requests_queue.append(Request(time=self.current_time))
            self._get_stats()
        while len(self.requests_queue) > 0 and len(self.free_channels) > 0:
            self._push_request()
        for req in self.requests_queue:
            req.tick_in_queue()
        for chn in self.working_channels:
            chn.tick()
        self.current_time += TICK_TIME

    def _push_request(self):
        channel = self.free_channels.pop(0)
        req = self.requests_queue.pop(0)
        channel.push(req)
        self.working_channels.append(channel)

    def _try_push_request(self, channel):
        if len(self.requests_queue) == 0:
            return False
        req = self.requests_queue.pop(0)
        channel.push(req)
        return True

    def _channel_finishing_callback(self, channel):
        if self._should_accept():
            self.accepted_requests += 1
            self.request_work_time.append(channel.request.working_time)
            self.request_queue_time.append(channel.request.in_queue_time)
            self.working_channels.remove(channel)
            self.free_channels.append(channel)
        else:
            self.rejected_requests += 1
            channel.push(channel.request)

    def _get_stats(self):
        self.avg_requests.append(len(self.requests_queue) + len(self.working_channels))
        self.busy_channels.append(len(self.working_channels))
        self.queueing_requests.append(len(self.requests_queue))

In [53]:
class Stats:
    def __init__(self, *, n, m, lmd, mu, p, smo, time):
        self.n = n
        self.m = m
        self.lmd = lmd
        self.mu = mu
        self.p = p
        self.q = 1 - p
        self.smo = smo
        self.time = time

        self._mu = mu * p
        self._ro = lmd / self._mu
        self._xi = self._ro / n

        self.p0 = self._t_p0()
        self.pk = [self.p0, *[self._t_p_k(k) for k in range(1, self.n + 1)]]
        self.pnr = [*self.pk, *[self._t_p_nr(r) for r in range(1, self.m + 1)]]

    def _t_p0(self):
        p0 = sum([(self._ro ** i) / factorial(i) for i in range(self.n + 1)])
        p0 += (self._ro ** (self.n + 1) / (self.n * factorial(self.n))) * ((1 - self._xi ** self.m) / (1 - self._xi))
        return 1 / p0

    def _t_p_k(self, k):
        return (self._ro ** k) / factorial(k) * self.p0

    def _t_p_nr(self, r):
        return (self._ro ** (self.n + r)) / ((self.n ** r) * factorial(self.n)) * self.p0

    def t_A(self):
        return self.lmd * (1 - self.pnr[self.n + self.m])
    def p_A(self):
        return Request.total_requests / self.time

    def t_Q(self):
        return 1 - self.pnr[self.n + self.m]
    def p_Q(self):
        return Request.total_requests / (Request.total_requests + self.smo.cancelled_requests)

    def t_p_Reject(self):
        return self.pnr[self.n + self.m]
    def p_p_Reject(self):
        return self.smo.cancelled_requests / (Request.total_requests + self.smo.cancelled_requests)

    def t_k(self):
        return self._ro * (1 - self.pnr[self.n + self.m])
    def p_k(self):
        return sum(self.smo.busy_channels) / self.time

    def t_r(self):
        r = ((self._ro ** (self.n + 1)) * self.p0) / (self.n * factorial(self.n))
        r *= (1 - (self.m + 1) * (self._xi ** self.m) + self.m * (self._xi ** (self.m + 1))) / ((1 - self._xi) ** 2)
        return r
    def p_r(self):
        return sum(self.smo.queueing_requests) / self.time

    def t_z(self):
        return self.t_r() + self.t_k()
    def p_z(self):
        return self.p_r() + self.p_k()

    def t_T_queue(self):
        return self.t_r() / self.t_A()
    def p_T_queue(self):
        return sum(self.smo.request_queue_time) / self.time

    def t_T_system(self):
        return self.t_z() / self.t_A()
    def p_T_system(self):
        return (sum(self.smo.request_queue_time) + sum(self.smo.request_work_time)) / self.time
    
    def p_p(self):
        d = defaultdict(int)
        for (i, j) in zip(self.smo.busy_channels, self.smo.queueing_requests):
            d[(i, j)] += 1
        ans = {}
        for (i, j) in d:
            ans[(i, j)] = d[(i, j)] / self.smo.current_time
        return ans

In [54]:
smo = SMO(
    n=N,
    m=M,
    lmd=LMD,
    mu=MU,
    p=P
)

while smo.current_time < 1000:
    smo.simulate()

In [55]:
stat = Stats(
    n=N,
    m=M,
    lmd=LMD,
    mu=MU,
    p=P,
    smo=smo,
    time=smo.current_time
)

$$ \bar\mu = \mu  p $$
$$ \rho = \frac{\lambda}{\bar\mu} $$
$$ \chi = \frac{\rho}{n}$$
$$ p_0 = [\sum_{i=0}^{n} \frac{\rho^i}{i!} +  \frac{(1-\chi^n) * \rho^{n+1}}{(1-\chi) * n * n!}]^{-1} $$
$$ p_k = \sum_{k=0}^{n} \frac{\rho^k * p_0}{k!} $$
$$ p_{n + r} = \sum_{r = 0}{m} \frac{\rho^{n + r} * p_0}{n^r * n!}$$

### Абсолютная пропускная способность

$$A = \lambda (1 - p_{n + m})$$ 

In [56]:
print(f"tA = {stat.t_A()}, pA = {stat.p_A()}")

tA = 1.0499999999999776, pA = 1.0679893201076154


### Относительная пропускная способность
$$ Q = 1 - p_{n + m} $$

In [57]:
print(f"tQ = {stat.t_Q()}, pQ = {stat.p_Q()}")

tQ = 0.09999999999999787, pQ = 0.1003570757376433


### Вероятность отказа
$$ P_{отк} = p_{n + m}$$

In [58]:
print(f"t_p_Reject = {stat.t_p_Reject()}, p_p_Reject = {stat.p_p_Reject()}")

t_p_Reject = 0.9000000000000021, p_p_Reject = 0.8996429242623567


### Среднее число занятых каналов
$$ k = \rho Q $$

In [59]:
print(f"tk = {stat.t_k()}, pk = {stat.p_k()}")

tk = 4.999999999999893, pk = 4.978950210501701


### Среднее число заявок в очереди
$$ r = \frac{\rho^{n + 1} * p_0 * (1 - (m + 1) * \chi^n + m * \chi^{m + 1}}{n * n! * (1 - \chi)^2} $$

In [60]:
print(f"tr = {stat.t_r()}, pr = {stat.p_r()}")

tr = 11.888888888889028, pr = 11.88088119119717


### Среднее число заявок в СМО
$$ z = r + k $$

In [61]:
print(f"tz = {stat.t_z()}, pz = {stat.p_z()}")

tz = 16.88888888888892, pz = 16.85983140169887


### Среднее время пребывания в очереди

$$ \tilde\tau_{оч} = r / A $$

In [62]:
print(f"t_T_Queue = {stat.t_T_queue()}, p_T_Queue = {stat.p_T_queue()}")

t_T_Queue = 11.322751322751696, p_T_Queue = 11.169458305425321


### Среднее время пребывания заявки в СМО
$$ \tilde\tau_{сист} = z / A$$

In [63]:
print(f"t_T_System = {stat.t_T_system()}, p_T_System = {stat.p_T_system()}")

t_T_System = 16.08465608465646, p_T_System = 16.122138778624336


In [64]:
df = pandas.DataFrame(
    {
        "busy_channels": smo.busy_channels[:10],
        "queueing requests": smo.queueing_requests[:10]
    }
)
df.describe()

,busy_channels,queueing requests
count,10.000000,10.00000
mean,4.500000,10.50000
std,1.581139,3.17105
min,0.000000,4.00000
25%,5.000000,12.00000
50%,5.000000,12.00000
75%,5.000000,12.00000
max,5.000000,12.00000


### Демонстрация установки стационарного режима

In [65]:
bcr.bar_chart_race(df, filename=None)

In [191]:
# probability_df = pandas.DataFrame(
#     {
#         "theoretical": [stat.pnr[i + j] for (i, j) in stat.p_p().keys()],
#         "practical": stat.p_p().values()
#     }
# )
# probability_df.T
print(f"t={stat.pnr[N + M]} p={stat.p_p()[(N, M)]}")

t=0.9000000000000021 p=0.9189908100926016


## Тест M >> N

In [211]:
N = 5
M = 100
LMD = 10
MU = 0.3
P = 0.8

Request.total_requests = 0
smo = SMO(
    n=N,
    m=M,
    lmd=LMD,
    mu=MU,
    p=P
)
while smo.current_time < 1000:
    smo.simulate()
    
stat = Stats(
    n=N,
    m=M,
    lmd=LMD,
    mu=MU,
    p=P,
    smo=smo,
    time=smo.current_time
)

print(f"tA = {stat.t_A()}, pA = {stat.p_A()}")
print(f"tQ = {stat.t_Q()}, pQ = {stat.p_Q()}")
print(f"t_p_Reject = {stat.t_p_Reject()}, p_p_Reject = {stat.p_p_Reject()}")
print(f"tk = {stat.t_k()}, pk = {stat.p_k()}")
print(f"tr = {stat.t_r()}, pr = {stat.p_r()}")
print(f"tz = {stat.t_z()}, pz = {stat.p_z()}")
print(f"t_T_Queue = {stat.t_T_queue()}, p_T_Queue = {stat.p_T_queue()}")
print(f"t_T_System = {stat.t_T_system()}, p_T_System = {stat.p_T_system()}")

tA = 1.1999999999999633, pA = 1.2769872301286749
tQ = 0.11999999999999633, pQ = 0.12831591639871381
t_p_Reject = 0.8800000000000037, p_p_Reject = 0.8716840836012861
tk = 4.999999999999848, pk = 4.982950170502104
tr = 99.86363636363636, pr = 99.17800821999361
tz = 104.8636363636362, pz = 104.16095839049572
t_T_Queue = 83.21969696969951, p_T_Queue = 93.1990880091973
t_T_System = 87.38636363636617, p_T_System = 98.15506844939664


## Тест N = M

In [206]:
N = 5
M = 5
LMD = 5
MU = 0.3
P = 0.7

Request.total_requests = 0
smo = SMO(
    n=N,
    m=M,
    lmd=LMD,
    mu=MU,
    p=P
)
while smo.current_time < 1000:
    smo.simulate()
    
stat = Stats(
    n=N,
    m=M,
    lmd=LMD,
    mu=MU,
    p=P,
    smo=smo,
    time=smo.current_time
)

print(f"tA = {stat.t_A()}, pA = {stat.p_A()}")
print(f"tQ = {stat.t_Q()}, pQ = {stat.p_Q()}")
print(f"t_p_Reject = {stat.t_p_Reject()}, p_p_Reject = {stat.p_p_Reject()}")
print(f"tk = {stat.t_k()}, pk = {stat.p_k()}")
print(f"tr = {stat.t_r()}, pr = {stat.p_r()}")
print(f"tz = {stat.t_z()}, pz = {stat.p_z()}")
print(f"t_T_Queue = {stat.t_T_queue()}, p_T_Queue = {stat.p_T_queue()}")
print(f"t_T_System = {stat.t_T_system()}, p_T_System = {stat.p_T_system()}")

tA = 1.0499799804034737, pA = 1.0579894201066078
tQ = 0.20999599608069475, pQ = 0.21330645161290324
t_p_Reject = 0.7900040039193053, p_p_Reject = 0.7866935483870968
tk = 4.999904668587971, pk = 4.965950340500391
tr = 4.734309774454721, pr = 4.717952820475402
tz = 9.734214443042692, pz = 9.683903160975792
t_T_Queue = 4.50895242082185, p_T_Queue = 4.15640843591877
t_T_System = 9.270857182726612, p_T_System = 9.122128778719107
